In [3]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

# Deep Learning Model

In [3]:
columns = ['ID', 'Text', 'Emotion', 'Confidence']
df_sad = pd.read_csv("./emotions/sad.txt", delimiter="\t", header=None, names=columns)
df_sad_test = pd.read_csv("./emotions/sad_test.txt", delimiter="\t", header=None, names=columns)

df_fear = pd.read_csv("./emotions/fear.txt", delimiter="\t", header=None, names=columns)
df_fear_test = pd.read_csv("./emotions/fear_test.txt", delimiter="\t", header=None, names=columns)

df_joy = pd.read_csv("./emotions/joy.txt", delimiter="\t", header=None, names=columns)
df_joy_test = pd.read_csv("./emotions/joy_test.txt", delimiter="\t", header=None, names=columns)

In [5]:
!pip install transformers

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python312\\Scripts\\f2py.exe' -> 'C:\\Python312\\Scripts\\f2py.exe.deleteme'




     ---------------------------------------- 0.0/130.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/130.7 kB ? eta -:--:--
     --- ------------------------------------ 10.2/130.7 kB ? eta -:--:--
     --- ------------------------------------ 10.2/130.7 kB ? eta -:--:--
     -------- ---------------------------- 30.7/130.7 kB 187.9 kB/s eta 0:00:01
     -------- ---------------------------- 30.7/130.7 kB 187.9 kB/s eta 0:00:01
     ----------------- ------------------- 61.4/130.7 kB 252.2 kB/s eta 0:00:01
     ------------------------------- ---- 112.6/130.7 kB 409.6 kB/s eta 0:00:01
     ------------------------------------ 130.7/130.7 kB 428.5 kB/s eta 0:00:00
  Using cached filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Using cached PyYAML-6.0.1-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
  Using cached regex-2023.12.25-cp312-cp312-win_amd64.whl.metadata (41 kB)
  Using cached tokeni

## Use BERT

In [4]:
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained("bert-base-uncased")

## Generate Encodings 

In [7]:
import torch

def predict_confidence(df):
    confidence_scores = []
    for i in range(len(df)):
        row = df.iloc[i]
        text = row['Text']
        encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            outputs = model(**encoded_input)
        confidence_score = torch.sigmoid(outputs.logits[0]).tolist()
        confidence_scores.append(confidence_score)

    return confidence_scores

In [5]:
confidence_scores_sad = predict_confidence(df_sad_test)
confidence_scores_fear = predict_confidence(df_fear_test)
confidence_scores_joy = predict_confidence(df_joy_test)

In [37]:
import numpy as np
mean_confidence_scores_sad = [np.mean(scores) for scores in confidence_scores_sad]
mean_confidence_scores_fear = [np.mean(scores) for scores in confidence_scores_fear]
mean_confidence_scores_joy = [np.mean(scores) for scores in confidence_scores_joy]

In [38]:
df_sad_test['Predicted_Confidence'] = mean_confidence_scores_sad
df_fear_test['Predicted_Confidence'] = mean_confidence_scores_fear
df_joy_test['Predicted_Confidence'] = mean_confidence_scores_joy

In [39]:
df_sad_test

,ID,Text,Emotion,Confidence,Predicted_Confidence
0,40860,My 2 teens sons just left in the car to get ha...,sadness,0.667,0.524193
1,40861,My 2 teens sons just left in the car to get ha...,sadness,0.458,0.523098
2,40862,HartRamsey'sUPLIFT If you're still discouraged...,sadness,0.396,0.502793
3,40863,@AmontanaW I nearly dropped my phone into the ...,sadness,0.271,0.525700
4,40864,Whenever I'm feeling sad I will listen to mons...,sadness,0.604,0.508531
...,...,...,...,...,...
668,41528,Why does Candice constantly pout #GBBO ðŸ’„ðŸ˜’,sadness,0.396,0.518122
669,41529,"@redBus_in #unhappy with #redbus CC, when I ta...",sadness,0.604,0.487128
670,41530,"@AceOperative789 no pull him afew weeks ago, s...",sadness,0.479,0.518037
671,41531,I'm buying art supplies and I'm debating how s...,sadness,0.375,0.512370


## Calculate MSE

In [40]:
from sklearn.metrics import mean_squared_error
mse_sad = mean_squared_error(df_sad_test['Confidence'], df_sad_test['Predicted_Confidence'])
mse_fear = mean_squared_error(df_fear_test['Confidence'], df_fear_test['Predicted_Confidence'])
mse_joy = mean_squared_error(df_joy_test['Confidence'], df_joy_test['Predicted_Confidence'])

In [41]:
print("MSE for sadness is: ", mse_sad)
print("MSE for fear is: ", mse_fear)
print("MSE for joy is: ", mse_joy)

MSE for sadness is:  0.04096111921848063
MSE for fear is:  0.04041703158624395
MSE for joy is:  0.04642148276791388
